# Regressione Lineare con un Dataset reale







## Il Dataset
Il [dataset per questo esercizio](https://developers.google.com/machine-learning/crash-course/california-housing-data-description) è basato sui dati del censimento del 1990 dalla California. Il dataset è vecchio ma fornisce ancora una grande opportunità per imparare il machine learning.

## Usa la versione giusta di TensorFlow

Il codice seguente fa si che questo Colab usi la versione più recente di TensorFlow.

In [ ]:
#@title Esegui su TensorFlow 2.x
%tensorflow_version 2.x

## Import di moduli necessari

In [ ]:
#@title Import dei moduli necessari
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt

# Le linee seguenti regolano la granularità del report.
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

## Il dataset

I Dataset sono spesso salvati o su disco o reperibili da uno specifico URL in [formato .csv](https://wikipedia.org/wiki/Comma-separated_values). 

Un .csv ben formato, contiene i nomi delle colonne nella prima riga, seguito da diverse righe di dati. Una virgola divide ogni valore in ogni riga. Per esempio, di seguito ci sono le prime cinque righe del file .csv:

```
"longitude","latitude","housing_median_age","total_rooms","total_bedrooms","population","households","median_income","median_house_value"
-114.310000,34.190000,15.000000,5612.000000,1283.000000,1015.000000,472.000000,1.493600,66900.000000
-114.470000,34.400000,19.000000,7650.000000,1901.000000,1129.000000,463.000000,1.820000,80100.000000
-114.560000,33.690000,17.000000,720.000000,174.000000,333.000000,117.000000,1.650900,85700.000000
-114.570000,33.640000,14.000000,1501.000000,337.000000,515.000000,226.000000,3.191700,73400.000000
```



### Caricare il file .csv in un pandas DataFrame

Questo Colab, come molti programmi di machine learning, reperiscono il file .csv e lo tengono in memoria come un DataFrame.

Il codice seguente importa il file .csv nel pandas DataFrame e scala i valori della colonna `median_house_value`:

In [ ]:
# Importa il Dataset
training_df = pd.read_csv(filepath_or_buffer="https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv")

# Scala i valori nella colonna median_house_value di un fattore 1000
training_df["median_house_value"] /= 1000.0

# Mostra le prime 5 righe del Dataframe
training_df.head()

Scalare i valori della colonna `median_house_value` mette il valore di ogni casa nell'ordine delle migliaia. Lo scaling manterrà i valori di loss e di learning rate in un range più amichevole.

Anche se scalare la colonna dei dati riferiti alle label è solitamente *non* essenziale, scalare i valori delle features in un modello con diverse feature *è* solitamente essenziale.


## Esamina il Dataset

Una fetta importante di molti progetti di machine learning è fare conoscenza con i tuoi dati. L'API pandas fornisce una funzione `describe` che da in output la seguente statistica riguardo ogni colonna nel DataFrame:

* `count`, che è il numero delle righe in quella colonna. Idealmente, `count` contiene lo stesso valore per ogni colonna.

* `mean` e `std`, che contiene la media e la  [deviazione standard](https://www.okpedia.it/deviazione-standard-scarto-quadratico-medio) (scarto quadratico medio) dei valori in ogni colonna.

* `min` e `max`, che contiene il valore minimo e massimo in ogni colonna.

* `25%`, `50%`, `75%`, che contengono diversi [quantili](https://developers.google.com/machine-learning/glossary/#quantile).



In [ ]:
# Ottieni le statistiche del Dataset
training_df.describe()


### Task 1: Identifica le anomalie nel Dataset

Vedi delle anomalie (valori strani) nel dataset?

Il valore massimo (max) di diverse colonne sembra essere molto alto comparato ai loro quantili. Per esempio, prendiamo la colonna total_rooms_column. Dati i valori dei quantili 25%, 50%, and 75%, ci aspettavamo di ottenere un valore per max intorno a 5,000 o 10,000. Tuttavia, il valore max è 37,937. Quando vedi delle anomalie in una colonna, sii attento nell'usare quella colonna come una feature. Detto questo, anomalie in feature potenziali alcune volte riflettono anomalie nella label, che potrebbe rendere la colonna (e la fa sembrare) una feature potente. Inoltre, come vedrai dopo nel corso, potresti essere in grado di rappresentare (pre-processare) i dati grezzi in modo da rendere le colonne utili come features.


## Definizione delle funzioni che costruiscono ed allenano un modello

Il seguente codice definisce due funzioni:
* `build_model(my_learning_rate)`, che costruisce un modello inizializzato in modo randomico.
* `train_model(model, feature, label, epochs)`, che allena un modello dai dati (feature e label) che passiamo


In [ ]:
#@title Definizione delle funzioni per costruire ed allenare un modello
def build_model(my_learning_rate):
  """Create and compile a simple linear regression model."""
  # Most simple tf.keras models are sequential.
  model = tf.keras.models.Sequential()

  # Describe the topography of the model.
  # The topography of a simple linear regression model
  # is a single node in a single layer.
  model.add(tf.keras.layers.Dense(units=1, 
                                  input_shape=(1,)))

  # Compile the model topography into code that TensorFlow can efficiently
  # execute. Configure training to minimize the model's mean squared error. 
  model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=my_learning_rate),
                loss="mean_squared_error",
                metrics=[tf.keras.metrics.RootMeanSquaredError()])

  return model        


def train_model(model, df, feature, label, epochs, batch_size):
  """Train the model by feeding it data."""

  # Feed the model the feature and the label.
  # The model will train for the specified number of epochs. 
  history = model.fit(x=df[feature],
                      y=df[label],
                      batch_size=batch_size,
                      epochs=epochs)

  # Gather the trained model's weight and bias.
  trained_weight = model.get_weights()[0]
  trained_bias = model.get_weights()[1]

  # The list of epochs is stored separately from the rest of history.
  epochs = history.epoch
  
  # Isolate the error for each epoch.
  hist = pd.DataFrame(history.history)

  # To track the progression of training, we're going to take a snapshot
  # of the model's root mean squared error at each epoch. 
  rmse = hist["root_mean_squared_error"]

  return trained_weight, trained_bias, epochs, rmse

print("Defined the create_model and traing_model functions.")

## Definizione delle funzioni per disegnare i grafici

In [ ]:
#@title Definizione delle funzioni per disegnare i grafici
def plot_the_model(trained_weight, trained_bias, feature, label):
  """Plot the trained model against 200 random training examples."""

  # Label the axes.
  plt.xlabel(feature)
  plt.ylabel(label)

  # Create a scatter plot from 200 random points of the dataset.
  random_examples = training_df.sample(n=200)
  plt.scatter(random_examples[feature], random_examples[label])

  # Create a red line representing the model. The red line starts
  # at coordinates (x0, y0) and ends at coordinates (x1, y1).
  x0 = 0
  y0 = trained_bias
  x1 = 10000
  y1 = trained_bias + (trained_weight * x1)
  plt.plot([x0, x1], [y0, y1], c='r')

  # Render the scatter plot and the red line.
  plt.show()


def plot_the_loss_curve(epochs, rmse):
  """Plot a curve of loss vs. epoch."""

  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Root Mean Squared Error")

  plt.plot(epochs, rmse, label="Loss")
  plt.legend()
  plt.ylim([rmse.min()*0.97, rmse.max()])
  plt.show()  

print("Defined the plot_the_model and plot_the_loss_curve functions.")

## Chiamata delle funzioni del modello
Una parte importante del machine learning è determinare quali [features](https://developers.google.com/machine-learning/glossary/#feature) correlare con le [label](https://developers.google.com/machine-learning/glossary/#label). Per esempio, i modelli di predizione del valore delle case nella realtà si basa tipicamente su centinaia di features e su features sintetiche. Tuttavia, questo modello fa affidamento solo su una feature. Per adesso, Scegliamo arbitrariamente `total_rooms` come feature.


In [ ]:
# I valori seguenti sono gli iperparametri
learning_rate = 0.01
epochs = 30
batch_size = 30

# Specifica la feature e la label
my_feature = "total_rooms" # Il numero totale di stanze in uno specifico isolato di città
my_label="median_house_value" # Il valore medio di una casa in uno specifico isolato di città.

#Cioè, andiamo a creare un modello che predice il valore di una casa basandosi soltanto su total_rooms.
# Scartiamo qualsiasi versione pre-esistente del modello.
my_model = None

# Invoca la funzione.
my_model = build_model(learning_rate)
weight, bias, epochs, rmse = train_model(my_model, training_df, 
                                         my_feature, my_label,
                                         epochs, batch_size)

print("\nThe learned weight for your model is %.4f" % weight)
print("The learned bias for your model is %.4f\n" % bias )

plot_the_model(weight, bias, my_feature, my_label)
plot_the_loss_curve(epochs, rmse)

Una certa quantità randomica entra in gioco ogni volta che alleni il modello. Conseguentemente, si avranno risultati differenti ogni volta. Detto questo, considerando il dataset e gli iperparametri, il modello farà generalmente un cattivo lavoro nel descrivere la relazione tra la feature selezionata e la label.

## Usa il modello per fare delle predizioni

Possiamo usare il modello allenato per fare delle predizioni. In pratica [si dovrebbero fare delle predizioni su dei dati che non sono stati usati in fase di allenamento](https://developers.google.com/machine-learning/crash-course/training-and-test-sets/splitting-data). Tuttavia, per questo esercizio, lavoreremo con un sottoinsieme dello stesso dataset di training. Nelle lezioni successive esploreremo dei modi per fare delle predizioni su dati non usati in fase di allenamento.

Prima di tutto, eseguiamo il codice per definire la funzione di predizione per le case:


In [ ]:
def predict_house_values(n, feature, label):
  """Predict house values based on a feature."""

  batch = training_df[feature][10000:10000 + n]
  predicted_values = my_model.predict_on_batch(x=batch)

  print("feature   label          predicted")
  print("  value   value          value")
  print("          in thousand$   in thousand$")
  print("--------------------------------------")
  for i in range(n):
    print ("%5.0f %6.0f %15.0f" % (training_df[feature][10000 + i],
                                   training_df[label][10000 + i],
                                   predicted_values[i][0] ))

Adesso, invoca la funzione di predizinoe su 10 dati:

In [ ]:
predict_house_values(10, my_feature, my_label)

### Task 2: Giudica il potere predittivo del modello

Guarda alla tabella precedente. Quano vicino è il valore predetto al valore della label? In altre parole, il nostro modello predice accuratamente il valore reale delle case?

In [ ]:
#@title Risposta.
#La maggior parte dei valori predetti differisce significativamente
#dal valore delle label, quindi il modello allenato probabilmente
#non ha molto potere predittivo. Tuttavia, i primi 10 dati potrebbero
#essere non rappresentativi rispetto l'intero dataset
 

## Task 3: Prova una feature diversa
La feature `total_rooms` ha poco potere predittivo. Una feature differente può portare ad un maggiore potere predittivo? Prova ad usare `population` come feature invece di `total_rooms`.

Nota: Quando cambi la feature, potresti avere bisogno di cambiare anche gli iperparametri.


In [ ]:
my_feature = "population"   # Rimpiazza il ? con population o possibilmente una colonna differente
                   

# Sperimenta con gli iperparametri.
learning_rate = 0.07
epochs = 100
batch_size = 70

# Non cambiare niente dopo questa linea.
my_model = build_model(learning_rate)
weight, bias, epochs, rmse = train_model(my_model, training_df, 
                                         my_feature, my_label,
                                         epochs, batch_size)
plot_the_model(weight, bias, my_feature, my_label)
plot_the_loss_curve(epochs, rmse)

predict_house_values(15, my_feature, my_label)

In [ ]:
#@title Double-click to view a possible solution.

my_feature = "population" # Pick a feature other than "total_rooms"

# Possibly, experiment with the hyperparameters.
learning_rate = 0.05
epochs = 18
batch_size = 3

# Don't change anything below.
my_model = build_model(learning_rate)
weight, bias, epochs, rmse = train_model(my_model, training_df, 
                                         my_feature, my_label,
                                         epochs, batch_size)

plot_the_model(weight, bias, my_feature, my_label)
plot_the_loss_curve(epochs, rmse)

predict_house_values(10, my_feature, my_label)

La feature `population` produce un risultato migliore rispetto `total_rooms`?

In [ ]:
#@title Risposta.

#L'allenamento non è interamente deterministico, ma la feature population tipicamente 
#converge ad un valore di Root Mean Square Error leggermente maggiore rispetto
#la feature total_rroms. Quindi, population appare essere allo stesso livello
#se non peggio di total_rooms nel fare le predizioni.


## Task 4: Definisci una feature sintetica

Abbiamo scoperto che sia `total_rooms` che `population` non risultano essere delle feature utili. Cioè, nè il numero totale delle stanze nel quartiere e nemmeno la popolazione nel quartiere sono state utili a predire il prezzo in media di una casa in quel quartiere. Forse però, il *rapporto* di `total_rooms` su `popolazione` potrebbe avere del potere predittivo. Cioè forse la densità in un isolato ha una qualche relazione con il valore di una casa.

Per esplorare questa ipotesi fai le seguente cose:

1. Crea una [feature sintetica](https://developers.google.com/machine-learning/glossary/#synthetic_feature) che è il rapporto tra `total_rooms` e `population`.

2. Aggiusta gli iperparametri.

3. Determina se questa feature sintetica produce dei valori di loss più bassi di qualsiasi altra singola feature che abbiamo provato in precedenza in questo esercizio.


In [ ]:
# Definisci una feature sintetica chiamata riins_per_person
training_df["rooms_per_person"] = training_df["total_rooms"]/training_df["population"] # Scrivi qui il tuo codice.

# Non cambiare la prossima riga
my_feature = "rooms_per_person"

# Assegna dei valori agli iperparametri
learning_rate = 0.5
epochs = 10
batch_size = 90

# Non cambiare niente sotto questa riga.
my_model = build_model(learning_rate)
weight, bias, epochs, rmse = train_model(my_model, training_df,
                                         my_feature, my_label,
                                         epochs, batch_size)

plot_the_loss_curve(epochs, rmse)
predict_house_values(15, my_feature, my_label)

In [ ]:
#@title Double-click to view a possible solution to Task 4.

# Define a synthetic feature
training_df["rooms_per_person"] = training_df["total_rooms"] / training_df["population"]
my_feature = "rooms_per_person"

# Tune the hyperparameters.
learning_rate = 0.06
epochs = 24
batch_size = 30

# Don't change anything below this line.
my_model = build_model(learning_rate)
weight, bias, epochs, mae = train_model(my_model, training_df,
                                        my_feature, my_label,
                                        epochs, batch_size)

plot_the_loss_curve(epochs, mae)
predict_house_values(15, my_feature, my_label)


Basandoci sui valori di loss, questa feature sintetica produce un modello migliore rispetto le singole feature che abbiamo provato nel Task 2  enel Task 3. Tuttavia, il modello non crea ancora delle predizioni buone.


## Task 5. Trova la feature (le features) i cui valori grezzi sono correlati con la label

Sino ad adesso, ci siamo basati su un approccio trial-error per identificare le possibili features per il modello. E' più saggio invece fare affidamento sulla statistica.

Una **matrice di correlazione** indica come ogni valore degli attributi grezzi sono correlati ai valori di altri attributi grezzi. I valori di correlazione hanno i seguenti significati:

* `1.0`: correlazione perfettamente positiva; ovvero, quando un attributo cresce, anche l'altro attributo cresce.

*`-1.0`: correlazione perfettamente negativa; ovvero, quando un attributo cresce, l'altro decresce.

*`0.0`: nessuna correlazione; le due colonne non sono  [linearmente dipendenti](https://en.wikipedia.org/wiki/Correlation_and_dependence#/media/File:Correlation_examples2.svg).

In generale, più è alto il valore della correlazione, più è grande il potere predittivo. Per esempio, un valore di correlazione di -0.8 implica un potere predittivo di gran lunga maggiore di una correlazione di -0.2 (si guarda in sostanza al valore assoluto della correlazione per giudicare il suo potere).

Il seguente codice genera la matrice di correlazione per gli attributi del Dataset delle case in california:


In [ ]:
# Genere una matrice di correlazione.
training_df.corr()

La matrice di correlazione mostra nove potenziali features (inclusa una sintetica) ed una label (`median_house_value`). Una correlazione fortemente negativa od una estremamente positiva con la label suggeriscono una geature potenzialmente molto buona.

**il tuo compito**: Determinare quali delle seguenti features potenziali appaiono essere le migliori candidate per essere assunte come features per il modello.


In [ ]:
#@title Risposta

#La feature 'median_income' si correla con lo 0.7 alla label median_house_value,
#quindi 'median_income potrebbe essere una buona feature'. Le altri sette features 
#potenziali hanno tutte una correlazione vicino allo 0.

#Se il tempo permette, prova la feature median_income e vedi come il modello migliora



Le matrici di correlazione non ci dicono tutta la storia. Nei seguenti esercizi, troveremo dei modi aggiuntivi per sbloccare il potere predittivo dalle feature potenziali.
